In [ ]:
%gui qt
import os, sys, re

from pyqtgraph.Qt import QtGui, QtCore
import pyqtgraph as pg
import pyqtgraph.dockarea as dock
import pyqtgraph.console as console
#import pyqtgraph.parametertree.parameterTypes as pTypes
#from pyqtgraph.parametertree import Parameter, ParameterTree, ParameterItem, registerParameterType

import numpy as np
import quaternion
import pandas as pd

import scipy.io.wavfile
import wavio   # works with NDI's broken .wav files
import pyaudio
import time

# Define classes and functions

In [ ]:
# These are the visualization classes

# TODO: right name for the classes?
class ChannelWidget(pg.GraphicsLayoutWidget):
# TODO: use this signal?
    cwsig_x_zoomed = QtCore.pyqtSignal()
    
    def __init__(self, data=None, rate=None, parent=None, **kargs):
        super(ChannelWidget, self).__init__(parent)
        self.data = None
        self.rate = None
        self.sec = None
        self.pen = (255,255,255,200)
        self.audioplot = self.addPlot(row=0)
        self.init_audioplot_data(data, np.int(rate))
# TODO: add spectrogram in second row
        self.parent = parent
        self.stream = self._open_stream_()
        self.selectors = [None, None]
        self.quickzoom_halfwin = 0.100

    def _open_stream_(self):
        '''Set up the audio stream for playback.'''
        self.pya = pyaudio.PyAudio()
        stream = self.pya.open(
            format = pyaudio.paInt16,
            channels = 1,
            rate = self.rate,
            output = True
        )
        return stream

    def init_audioplot_data(self, data, rate):
        '''Clear existing plots and load new audio.'''
        self.data = data
        self.rate = rate
        self.sec = np.arange(len(data)) / rate
        self.audioplot.plot(
            x=self.sec,
            y=data,
            pen=self.pen,
            clear=True
        )
        self.audioplot.setDownsampling(auto=True)
# TODO: emit signal when data changes (or determine which signal is already emitted)
# TODO: set to full zoom out when data changes

    def zoom_to_selectors(self):
        '''Zoom viewbox to bounds selected by selectors.'''
        try:
            self.audioplot.getViewBox().setXRange(
                self.selectors[0].value(),
                self.selectors[1].value()
            )
        except AttributeError: # selectors[0] or selectors[1] is None
            pass

    def play_viewbox(self):
        '''Play the audio currently displayed in the viewbox.'''
        xrng = np.array(self.audioplot.getViewBox().viewRange()[0])
        s0, s1 = (xrng * rate).astype(np.int)
        self.play_samples(s0, s1)
        
    def play_all(self):
        '''Play all audio.'''
        self.play_samples(0, len(self.data) - 1)

    def play_samples(self, s0, s1):
        '''Play audio from sample s0 to sample s1.'''
# TODO: support other dtype besides int16
        self.stream.write(
            self.data[s0:s1].astype(np.int16).tostring()
        )

    def mousePressEvent(self, e):
        self._pressed_screenpos = e.screenPos()
        super(ChannelWidget, self).mousePressEvent(e)
        
    def mouseReleaseEvent(self, e):
        xpos = self.audioplot.vb.mapSceneToView(e.pos()).x()
        did_not_pan = e.screenPos() == self._pressed_screenpos        
        cidx = 1 if e.modifiers() == QtCore.Qt.ShiftModifier else 0
        if e.button() == QtCore.Qt.LeftButton and did_not_pan:
            if self.selectors[cidx] is None:
                self.selectors[cidx] = pg.InfiniteLine(xpos, movable=True)
                self.audioplot.addItem(self.selectors[cidx])
            if xpos >= 0.0:
# TODO: check that we don't go past the end also
                self.selectors[cidx].setValue(xpos)
# TODO: keep quickzoom behavior?
            if e.modifiers() == QtCore.Qt.ControlModifier:
                self.audioplot.getViewBox().setXRange(
                    xpos - self.quickzoom_halfwin,
                    xpos + self.quickzoom_halfwin
                )
            try:
                self.selectors.sort(key=lambda x: x.value())
            except AttributeError:  # a selector is not yet set
                pass
        e.accepted = True  # TODO: useful/needed?
        super(ChannelWidget, self).mouseReleaseEvent(e)

class ArticuWidget(pg.GraphicsLayoutWidget):
    '''Widget that encapsulates element-based articulatory data, e.g. EMA,
x-ray microbeam.'''
    def __init__(self, df, landmarkdf, tongue, parent, **kargs):
        super(ArticuWidget, self).__init__(parent)
        self.df = df
        self.landmarkdf = landmarkdf
        self.tongue = tongue   # List of element names.
        self.pen = (255,255,255,200)
        self.plots = []
        self.elements = [
            el.replace('_x', '') for el in df.columns if el.endswith('_x')
        ]
        self._sel_t1 = None
        self._sel_t2 = None
        self._sel_df = None
        self._sel_landmarkdf = None
        self._sel_element_cols_x = None
        self._sel_element_cols_y = None
        self._sel_tongue_cols_x = None
        self._sel_tongue_cols_y = None
# TODO: don't hardcode pos_vel_dim
        self.pos_vel_dim = 'x'
        self.pos_vel_elements = []
        self.minsymbsize = 1   # Minimum symbol size
        self.maxsymbsize = 5   # Maximum symbol size
        self.minalpha = 1      # Minimum alpha
        self.maxalpha = 255    # Maximum alpha
        self.frameplot = self.addPlot()  # Plot of a single frame
        self.traceplot = self.addPlot()  # Plot of time trace
        self.posplot = self.addPlot()    # Plot of element positions over time
        self.velplot = self.addPlot()    # Plot of element velocities over time
        self.vel_selector = pg.InfiniteLine()
        self.pos_selector = pg.InfiniteLine()
        
    def tselect(self, t1, t2):
        '''Select a time range from dataframes and cache.'''
        self._sel_t1 = t1
        self._sel_t2 = t2
        xmask = (self.df.sec >= t1) & (self.df.sec <= t2)
        if not xmask.any():  # Zero length region is selected.
            # Select row nearest xend.
            xmask[(self.df.sec - t2).abs().argmin()] = True
            minsymbsize = self.maxsymbsize
            minalpha = self.maxalpha
        else:
            minsymbsize = self.minsymbsize
            minalpha = self.minalpha
        symbsizes = np.linspace(minsymbsize, self.maxsymbsize, num=xmask.sum())
        alphas = np.linspace(minalpha, self.maxalpha, num=xmask.sum())
        mskdf = self.df.loc[xmask, :].copy()
        mskdf = mskdf.assign(symbsizes=symbsizes)
        self._sel_df = mskdf
        self._sel_element_cols_x = [
            '{:}_x'.format(el) for el in self.elements
        ]
        self._sel_element_cols_y = [
            '{:}_y'.format(el) for el in self.elements
        ]
        self._sel_tongue_cols_x = [
            '{:}_x'.format(el) for el in self.tongue
        ]
        self._sel_tongue_cols_y = [
            '{:}_y'.format(el) for el in self.tongue
        ]
        
    def tplot(self, t1, t2):
        '''Create plots for time range.'''
        if t1 != self._sel_t1 and t2 != self._sel_t2:
            self.tselect(t1, t2)
        self.frameplot.clear()
        self.traceplot.clear()
        self.posplot.clear()
        self.velplot.clear()
        elemdims = [
            '{}_{}'.format(el, self.pos_vel_dim) for el in self.pos_vel_elements
        ]
        for pd in elemdims:
            print(pd)
            self.posplot.plot(
                self._sel_df.sec.values,
                self._sel_df.loc[:, [pd]].values.squeeze(),
                symbol='o',
                pen=None,
                symbolBrush=(0, 0, 128, 128),
                symbolSize=self.maxsymbsize
            )
            self.posplot.addItem(self.pos_selector)
            self.velplot.plot(
                self._sel_df.sec.values,
                self._sel_df.loc[:, [pd + '_vel']].values.squeeze(),
                symbol='o',
                pen=None,
                symbolBrush=(0, 0, 128, 128),
                symbolSize=self.maxsymbsize
            )
            self.velplot.addItem(self.vel_selector)
        if self.landmarkdf is not None:
            for g in self.landmarkdf.groupby('landmark'):
                self.frameplot.plot(g[1].x, g[1].y, pen=self.pen)
                self.traceplot.plot(g[1].x, g[1].y, pen=self.pen)
        self.update_tplot(t1, t1)   # Set to start of frame

    def update_tplot(self, ut1, ut2):
        global mskdf, xmask
        '''Update existing tplot between ut1 and ut2.'''
        #cw.audioplot.dataItems[0].setData(cw.data[::-2])
# TODO: throw an error if ut1:ut2 not bounded by t1:t2
        self.pos_selector.setValue(ut2)
        self.vel_selector.setValue(ut2)
        xmask = (self._sel_df.sec >= ut1) & (self._sel_df.sec <= ut2)
# TODO: is choosing first row right solution for all-false xmask?
        if not xmask.any():
            xmask.iloc[0] = True
        mskdf = self._sel_df.loc[xmask, :]
        endidx = xmask[::-1].argmax()  # index of last selected value
        try:
            self.frameplot.dataItems[2].setData(
                mskdf.loc[endidx, self._sel_tongue_cols_x].values,
                mskdf.loc[endidx, self._sel_tongue_cols_y].values
            )
        except IndexError:
            self.frameplot.plot(  # Plot tongue contour at end of selection.
                mskdf.loc[endidx, self._sel_tongue_cols_x].values,
                mskdf.loc[endidx, self._sel_tongue_cols_y].values,
                pen=self.pen
            )
        try:
            self.frameplot.dataItems[3].setData(
                mskdf.loc[endidx, self._sel_element_cols_x].values,
                mskdf.loc[endidx, self._sel_element_cols_y].values
            )
        except IndexError:
            self.frameplot.plot(  # Plot non-tongue elements at end of selection.
                mskdf.loc[endidx, self._sel_element_cols_x].values,
                mskdf.loc[endidx, self._sel_element_cols_y].values,
                symbol='o',
                pen=None,
                symbolBrush=(0, 0, 128, 128),
                symbolSize=self.maxsymbsize
            )
        for idx, pts in enumerate(
                zip(self._sel_element_cols_x, self._sel_element_cols_y)
            ):
            elx, ely = pts
# TODO: don't hardcode '_x'
            if elx.replace('_x', '') in self.tongue:
                symbr = (128, 128, 128, 128)
            else:
                symbr = (0, 0, 128, 128)
            myidx = 4 + idx
            try:
                self.frameplot.dataItems[myidx].setData(
                    mskdf.loc[:endidx, elx].values,
                    mskdf.loc[:endidx, ely].values
                )
            except IndexError:
                self.traceplot.plot(
                    mskdf.loc[:endidx, elx].values,
                    mskdf.loc[:endidx, ely].values,
                    symbolSize=mskdf.loc[:endidx, 'symbsizes'],
                    pen=None,
                    symbol='o',
                    symbolBrush=symbr
                )
        pg.QtGui.QApplication.processEvents()  # Force a redraw.


In [ ]:
#df = pd.read_csv('/media/sf_EMA-ECOG/pilot-subjects/Subject_3/SN3_COMMA_01.ndi', sep='\t')
to_drop = [c for name in ['EMPTY', 'REF_', 'UNK'] for c in df.columns if c.startswith(name)]
#df = df.drop(to_drop, axis=1)
#df
to_drop

In [ ]:
# These are functions that are specific to the EMA data and go in a separate repo.
def ecog_speakerdir_for_speaker(speaker):
    '''Convert speaker names like SN125 to speaker directories like 
Subject_125.'''
    m = re.search(r'(\d+)$', speaker)
    speakerdir = 'Subject_{}'.format(m.groups()[0])
    return speakerdir

def read_ecog_speaker_audio(basepath, speaker, dataname, rep):
    '''Read a UCSF EMA (ECOG) speaker audio file. Return sample rate and
audio data as a numpy array.
'''
    if not isinstance(rep, str):  # if rep is passed as an int
        rep = '{:03d}'.format(rep)
    fname = os.path.join(
        basepath,
        ecog_speakerdir_for_speaker(speaker),
        '{}_{}_{}.wav'.format(speaker, dataname, rep)
    )
    # Use wavio for broken .wav files
    w = wavio.read(fname)
    return (w.rate, w.data[:, 0])
#    return scipy.io.wavfile.read(fname)
    
def read_ecog_speaker_data(basepath, speaker, dataname, rep, drop_prefixes=['EMPTY']):
    '''Read a UCSF EMA (ECOG) speaker data file into a DataFrame.
The directory name is formed from basepath and speaker.
The filename is formed from speaker, dataname, and the repetition (rep). The
rep parameter can be a string or an integer.
Empty columns (identified by empty or whitespace-only column names) are dropped.
'''
    snre = re.compile(r'\d+$')
    if not isinstance(rep, str):  # if rep is passed as an int
        rep = '{:03d}'.format(rep)
    fname = os.path.join(
        basepath,
        ecog_speakerdir_for_speaker(speaker),
        '{}_{}_{}.ndi'.format(speaker, dataname, rep)
    )
    
    df = pd.read_csv(fname, sep='\t')
    to_drop = [c for name in drop_prefixes for c in df.columns if c.startswith(name)]
    df = df.drop(to_drop, axis=1)

    df = df.rename(columns={'time': 'sec'})

    # Calculate velocities for all coordinate columns and add as <coordinate>_vel columns.
    coordcols = [
        c for c in df.columns if c[-2:] in ['_x', '_y', '_z']
    ]
    df = df.join(df[coordcols].diff(), rsuffix='_vel')

#    return with_quats(df, sensors)
    return df

def read_marquette_speaker_data(basepath, speaker, dataname):
    '''Read Marquette EMA speaker data from a directory.'''
    spkpath = os.path.join(basepath, speaker)

    sensors = ["REF","TD","TL","TB","UL","LL","LC","MI","PL","OS","MS","UNK0","UNK1"]
    subcolumns = ["ID","Status","x","y","z","q0","qx","qy","qz"]
    better_head = \
        ['sec', 'measid', 'wavid'] + \
        ['{}_{}'.format(s, c) for s in sensors for c in subcolumns]
    coordcols = [
        '{}_{}'.format(s, d) for s in sensors for d in ['x', 'y', 'z']
    ]

    read_csv_kwargs = dict(
        sep='\t',
        header=None,            # The last three parameters
        skiprows=1,             # are used to override
        names=better_head       # the existing file header.
    )

    datadf = pd.read_csv(
        os.path.join(spkpath, 'Data', '{}_{}.tsv'.format(speaker, dataname)),
        **read_csv_kwargs
    )
    # Calculate velocities for all coordinate columns and add as <coordinate>_vel columns.
    datadf = datadf.join(datadf[coordcols].diff(), rsuffix='_vel')

    paldf = pd.read_csv(
        os.path.join(
            spkpath, 'Calibration', 'Palate',
            '{}_palatetrace.tsv'.format(speaker)
        ),
        **read_csv_kwargs
    )

    bpdf = pd.read_csv(
        os.path.join(
            spkpath, 'Calibration', 'Biteplate',
            '{}_Biteplate.tsv'.format(speaker)
        ),
        **read_csv_kwargs
    )

    rotdf = pd.read_csv(
        os.path.join(
            spkpath, 'Calibration', 'Biteplate',
            '{}_Biteplate_Rotation.txt'.format(speaker)
        ),
        sep='\t',
        header=None
    )
    
#    return (with_quats(datadf, sensors), with_quats(paldf, sensors),
#            with_quats(bpdf, sensors), rotdf)
    return (datadf, paldf, bpdf, rotdf)


In [ ]:
# These are functions that are specific to the xray data and go in a separate repo.
def walk_xray_datadir(datadir):
    '''Walk datadir and return a dict in which the keys are speakers and the
values are lists of their utterances.'''
    speakers = {}
    jwre = re.compile(r'(JW\d+)$')
    for root, dirnames, filenames in os.walk(datadir):
        m = jwre.search(root)
        if m:
            spkr = m.groups()[0]
            utterances = [
                f.replace('.txy', '') for f in filenames if f.endswith('.txy')
            ]
            speakers[spkr] = utterances
    return speakers

def load_xray_files(datadir, speaker, utterance, badval=1000000):
    '''Load files from xray database related to a speaker and utterance.
Return as DataFrames. Convert the time data to seconds and distance
measurements to mm. Also remove bad values (1000000).'''
    spkrpath = os.path.join(datadir, speaker)
    rate, au = scipy.io.wavfile.read(os.path.join(spkrpath, utterance + '.wav'))

    # Load the tongue data
    articfile = os.path.join(spkrpath, utterance + '.txy')
    coordcols = [
        'UL_x', 'UL_y', 'LL_x', 'LL_y', 'T1_x', 'T1_y', 'T2_x', 'T2_y',
        'T3_x', 'T3_y', 'T4_x', 'T4_y', 'MI_x', 'MI_y', 'MM_x', 'MM_y'
    ]
    articdf = pd.read_csv(
            articfile,
            sep='\t',
            na_values=badval,
            names=['sec'] + coordcols
    )
    articdf['sec'] *= 1e-6 # Convert to seconds

    # Load the palate data.
    palfile = os.path.join(spkrpath, 'PAL.DAT')
    paldf = pd.read_csv(palfile, sep='\s+', header=None, names=['x', 'y'])

    # Load the pharynx data.
    phafile = os.path.join(spkrpath, 'PHA.DAT')
    phadf = pd.read_csv(phafile, sep='\s+', header=None, names=['x', 'y'])

    landmarkdf = pd.concat([
        paldf.assign(landmark = pd.Series(['palate'] * len(paldf))),
        phadf.assign(landmark = pd.Series(['pharynx'] * len(phadf)))
    ])
    
    # Convert all coordinates to mm.
    articdf[coordcols] *= 1e-3
    landmarkdf.loc[:, ['x','y']] *= 1e-3

    # Calculate velocities for all coordinate columns and add as <coordinate>_vel columns.
    articdf = articdf.join(articdf[coordcols].diff(), rsuffix='_vel')
    
    return (rate, au, articdf, landmarkdf)

In [ ]:
# These are functions that might go in an app that uses the visualization classes.
# Currently unused.
def speaker_selected():
    spkr = spkrcbox.currentText()
    while uttcbox.count() > 0:
        uttcbox.removeItem(0)
    uttcbox.addItem('')
    for utt in speakers[spkr]:
        uttcbox.addItem(utt)

def utterance_selected(self):
    spkr = spkrcbox.currentText()
    utt = uttcbox.currentText()
    load_files(spkr, utt)
    plot_new_audio()

def speaker_combobox():
    '''Return a combobox with speaker names.'''
    cb = QtGui.QComboBox()
    cb.addItem('')
    spkrlist = sorted(speakers.keys())
    for spkr in spkrlist:
        cb.addItem(spkr)
    return cb

def pellet_group():
    '''Return a QGroupBox filled with pellet checkboxes.'''
    gb = QtGui.QGroupBox()
    vbox = QtGui.QVBoxLayout();
    cb = QtGui.QComboBox()
    cb.addItem('y')
    cb.addItem('x')
    vbox.addWidget(cb)
    for pellet in ['UL','LL', 'T1','T2', 'T3', 'T4', 'MI', 'MM']:
        checkbox = QtGui.QCheckBox(pellet)
        vbox.addWidget(checkbox)
    gb.setLayout(vbox)
    return gb

def get_selected_pellet_dims():
    '''Return a list of <pellet><dim> strings that are selected.'''    
    dim = pelletgroup.findChild(QtGui.QComboBox).currentText()
    pd = []
    for cb in pelletgroup.findChildren(QtGui.QCheckBox):
        if cb.isChecked():
            pd.append(cb.text() + dim)
    return pd
        

## Make some app functions

In [ ]:
def app_make_tplot(e):
    '''Handle a zoom event in the audio and pass it to the articulation.'''
    tstart, tend = e.viewRange()[0]
    aw.tplot(tstart, tend)

# Work with some data

## EMA-ECOG data

In [ ]:
# Load EMA-ECOG data. You can specify the repetition as a literal string or integer.

ecogspeaker = 'SN3'
ecogbase = '/media/sf_EMA-ECOG/pilot-subjects'

rate, au = read_ecog_speaker_audio(ecogbase, ecogspeaker, 'comma', '01')
# Remove all columns that begin with an element in drop_prefixes
datadf = read_ecog_speaker_data(
    ecogbase, ecogspeaker, 'comma', '01',
    drop_prefixes=['EMPTY', 'REF', 'UNK']
)
#bpdf = read_ecog_speaker_data(ecogbase, ecogspeaker, 'biteplate', 0)
#paldf = read_ecog_speaker_data(ecogbase, ecogspeaker, 'palate_trace', 1)

In [ ]:
aw.update_tplot(0.0, 0.2)

In [ ]:
# Make widgets for audio channel and articulation data. Hook them together so that
# when the xrange changes on the audio channels the articulation windows update.
cw = ChannelWidget(au, rate)
cw.show()
aw = ArticuWidget(datadf, None, tongue=['TB', 'TD', 'TL'], parent=None)
# TODO: create polygon from 'LL', to 'LC', to 'UL'
# TODO: xyz map
# TODO load .ndi instead of .tsv file and create list of pellets from the header
aw.pos_vel_elements = ['LL', 'UL']
aw.pos_vel_dim = 'y'
aw.tplot(0.0, 0.25)
cw.audioplot.sigXRangeChanged.connect(app_make_tplot)
aw.show()

In [ ]:
aw.update_tplot(1.0, 1.2)

In [ ]:
# Get rotations
m_bpdf_rot = get_rotation(m_bpdf)

## X-ray microbeam data

In [ ]:
# rate = audio sample rate
# au = audio data
# articdf = .txy data loaded into a dataframe
#    format: a time column labelled 'sec' + columns named <element>_x|y
# landmarkdf = PAL.DAT and PHA.DAT loaded into a dataframe
#    format: columns 'x' and 'y' + 'landmark' with values palate|pharynx

datadir = '/home/ubuntu/src/xray_microbeam_db/app/data/'

rate, au, articdf, landmarkdf = load_xray_files(datadir, 'JW11', 'tp001')

In [ ]:
# Make widgets for audio channel and articulation data. Hook them together so that
# when the xrange changes on the audio channels the articulation windows update.
cw = ChannelWidget(au, rate)
cw.show()
aw = ArticuWidget(articdf, landmarkdf, tongue=['T1', 'T2', 'T3', 'T4'], parent=None)
#aw.tplot(1.0, 2.0)
cw.audioplot.sigXRangeChanged.connect(app_make_tplot)
aw.show()

## Unused cells

The following cells were executed during development. We'll keep them around for a while in case they are useful for reference.

In [ ]:
pq.setData(cw.data[::-1])
QtGui.QApplication.processEvents()

In [ ]:
cw.audioplot.dataItems[0].setData(cw.data[::-2])

In [ ]:
cw.audioplot.sigXRangeChanged

In [ ]:
pq = cw.audioplot.plot(cw.sec, cw.data[::-1])

In [ ]:
cw.init_audioplot_data(cw.data[::-2], cw.rate)

In [ ]:
cw = ChannelWidget(au, rate)
cw.show()
# Change viewbox based on selectors
#cw.zoom_to_selectors()
# Play audio based on current viewbox
#cw.play_viewbox()
# Replace existing plot data
#cw.init_audioplot_data(cw.data[::-1], cw.rate)

In [ ]:
cw.play_viewbox()

In [ ]:
def updatePlots():
    xstart, xend = plots['audio'].getViewBox().viewRange()[0]
    minsymbsize = 1   # Minimum symbol size
    maxsymbsize = 5   # Maximum symbol size
    minalpha = 1      # Minimum alpha
    maxalpha = 255    # Maximum alpha
    pen = (255, 255, 255, 200)
    xmask = (tngdf.sec >= xstart) & (tngdf.sec <= xend)
    if not xmask.any():  # Zero length region is selected.
        # Select row nearest xend.
        xmask[(tngdf.sec - xend).abs().argmin()] = True
        minsymbsize = maxsymbsize
        minalpha = maxalpha
    symbsizes = np.linspace(minsymbsize, maxsymbsize, num=xmask.sum())
    alphas = np.linspace(minalpha, maxalpha, num=xmask.sum())
    mskdf = tngdf.loc[xmask, :].copy()
    mskdf = mskdf.assign(symbsizes=symbsizes)
    brushes = [pg.mkBrush(255, 255, 255, a) for a in alphas]
    #mskdf = mskdf.assign(tbrushes=[pg.mkBrush(255, 255, 255, a) for a in alphas])
    #mskdf = mskdf.assign(othbrushes=[pg.mkBrush(255, 255, 255, a) for a in alphas])
    #mskdf = mskdf.assign(othbrushes=[pg.mkBrush(0, 0, 255, a) for a in alphas])
    xpts = ['T1x', 'T2x', 'T3x', 'T4x', 'ULx', 'LLx', 'MIx']
    ypts = ['T1y', 'T2y', 'T3y', 'T4y', 'ULy', 'LLy', 'MIy']

    plots['pos'].clear()
    plots['vel'].clear()
    for pd in get_selected_pellet_dims():
        plots['pos'].plot(
            mskdf.sec.values,
            mskdf.loc[:, [pd]].values.squeeze(),
            symbol='o',
            pen=None,
            symbolBrush=(0, 0, 128, 128),
            symbolSize=maxsymbsize
        )
        plots['pos'].addItem(poscursor)
        plots['vel'].plot(
            mskdf.sec.values,
            mskdf.loc[:, [pd + '_vel']].values.squeeze(),
            symbol='o',
            pen=None,
            symbolBrush=(0, 0, 128, 128),
            symbolSize=maxsymbsize
        )
        plots['vel'].addItem(velcursor)
# TODO: make granularity configurable
    granularity = 1
    to_show_idx = mskdf.index[::granularity]
    if mskdf.index[-1] not in to_show_idx:
        to_show_idx.append(msdkdf.index[-1])
    for idx in to_show_idx:
        plots['static'].plot(paldf.x, paldf.y, pen=pen, clear=True)
        plots['static'].plot(phadf.x, phadf.y, pen=pen)
        plots['static'].plot(  # Plot tongue contour at end of selection.
            mskdf.loc[idx, xpts[:4]].values,
            mskdf.loc[idx, ypts[:4]].values,
            pen=pen
        )
        plots['static'].plot(  # Plot non-tongue elements at end of selection.
            mskdf.loc[idx, xpts[4:7]].values,
            mskdf.loc[idx, ypts[4:7]].values,
            symbol='o',
            pen=None,
            symbolBrush=(0, 0, 128, 128),
            symbolSize=maxsymbsize
        )
        plots['trajec'].plot(paldf.x, paldf.y, pen=pen, clear=True)
        plots['trajec'].plot(phadf.x, phadf.y, pen=pen)
        for elx, ely in zip(xpts, ypts):
            if elx.startswith('T'):
                symbr = (128, 128, 128, 128)
            else:
                symbr = (0, 0, 128, 128)
            plots['trajec'].plot(
                mskdf.loc[:idx, elx].values,
                mskdf.loc[:idx, ely].values,
                symbolSize=mskdf.loc[:idx, 'symbsizes'],
                pen=None,
                symbol='o',
                symbolBrush=symbr
            )
        poscursor.setPos(mskdf.loc[idx, 'sec'])
        velcursor.setPos(mskdf.loc[idx, 'sec'])
        pg.QtGui.QApplication.processEvents()  # Force a redraw.

In [ ]:
#QtGui.QApplication.setGraphicsSystem('raster')
app = QtGui.QApplication([])
mw = QtGui.QMainWindow()
mw.resize(800,700)
mw.setWindowTitle('xrayvis')
area = dock.DockArea()
mw.setCentralWidget(area)
pg.setConfigOptions(antialias=True) # Enable antialiasing for prettier plots

ctrls = dock.Dock('Controls', size=(1,1))
audiodock = dock.Dock('Audio')
staticdock = dock.Dock('Static trace')
trajecdock = dock.Dock('Trajectories')
posdock = dock.Dock('Postition over time')
veldock = dock.Dock('Velocity over time')
cnsldock = dock.Dock('Console')
area.addDock(ctrls, 'left')
area.addDock(audiodock, 'right', ctrls)
area.addDock(staticdock, 'bottom', audiodock)
area.addDock(trajecdock, 'right', staticdock)
area.addDock(posdock, 'bottom', staticdock)
area.addDock(veldock, 'bottom', trajecdock)
area.addDock(cnsldock, 'top', audiodock)
playall = QtGui.QPushButton('Play all')
playsel = QtGui.QPushButton('Play sel')
anim = QtGui.QPushButton('Animate')
ctrls.addWidget(playall, row=0, col=0)
ctrls.addWidget(playsel, row=1, col=0)
ctrls.addWidget(anim, row=2, col=0)

plots['audio'] = pg.PlotWidget(title="Audio")
plots['static'] = pg.PlotWidget(title="Static trace")
plots['trajec'] = pg.PlotWidget(title="Trajectories")
plots['pos'] = pg.PlotWidget(title="Position over time")
plots['vel'] = pg.PlotWidget(title="Velocity over time")

# Add cursors.
poscursor = pg.InfiniteLine()
velcursor = pg.InfiniteLine()

# Set up namespace for use in the console.
namespace = {
    'pg': pg, 'np': np, 'pd': pd, 'au': au, 'rate': rate,
    'timepts': timepts, 'tngdf': tngdf, 'paldf': paldf, 'phadf': phadf,
    'plots': plots, 'app': app, 'mw': mw, 'area': area
}
cnsltext = 'Variables: {:}'.format(", ".join(namespace.keys()))
plots['cnsl'] = console.ConsoleWidget(namespace=namespace, text=cnsltext)
plots['cnsl'].show()

audiodock.addWidget(plots['audio'])
staticdock.addWidget(plots['static'])
trajecdock.addWidget(plots['trajec'])
posdock.addWidget(plots['pos'])
veldock.addWidget(plots['vel'])
cnsldock.addWidget(plots['cnsl'])

# Set up speaker/utterance menu.
speakers = walk_datadir()
spkrcbox = speaker_combobox()
uttcbox = QtGui.QComboBox()
pelletgroup = pellet_group()
ctrls.addWidget(spkrcbox, row=3, col=0)
ctrls.addWidget(uttcbox, row=4, col=0)
ctrls.addWidget(pelletgroup, row=5, col=0)

plots['audio'].getPlotItem().ctrlMenu.addAction(spkract)
plots['audio'].getPlotItem().ctrlMenu.addAction(uttact)

# Hook up signals and slots.
playall.clicked.connect(play_all)
playsel.clicked.connect(play_sel)
anim.clicked.connect(updatePlots)
spkrcbox.activated.connect(speaker_selected)
uttcbox.activated.connect(utterance_selected)
#spkract.defaultWidget().activated.connect(speaker_selected)
#uttact.defaultWidget().activated.connect(utterance_selected)

mw.show()

## Start Qt event loop unless running in interactive mode or using pyside.
if __name__ == '__main__':
    import sys
    if (sys.flags.interactive != 1) or not hasattr(QtCore, 'PYQT_VERSION'):
        QtGui.QApplication.instance().exec_()